# UFF vs SPARC — Guided Notebook

Load a SPARC-like CSV, implement your UFF model in `uff_model.py`, and fit.

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from uff_model import v_circ_uff
from analyze_sparc import load_sparc_csv, posterior_logprob, mh_sampler, aic_bic, summarize_chain

data_path = Path("data/DEMO_GALAXY.csv")  # change me
df = load_sparc_csv(str(data_path))
df.head()


## Set Priors
Define explicit, wide-but-sane priors for your parameters (edit names to match your UFF).

In [ ]:

priors = {
    "V0": (50.0, 350.0),
    "Rc": (0.1, 30.0),
    "beta": (-1.0, 2.0),
}
param_names = list(priors.keys())
theta0 = np.array([(priors[n][0]+priors[n][1])/2 for n in param_names])
theta0


## Likelihood and Posterior

In [ ]:

R = df["R_kpc"].values
V = df["V_obs_kms"].values
eV = df["e_V_kms"].values

def lp(theta):
    return posterior_logprob(theta, R, V, eV, df, priors)

lp(theta0)


## Run MCMC (Metropolis–Hastings)

In [ ]:

chain, acc = mh_sampler(theta0, lp, steps=20000, step_scale=0.10, burn=5000, thin=10, random_state=123)
acc


## Summaries and AIC/BIC

In [ ]:

stats = summarize_chain(chain, param_names)
idx = np.argmax(chain[:, -1])
theta_map = chain[idx, :len(param_names)]
logpost_max = chain[idx, -1]
AIC, BIC = aic_bic(logpost_max, k=len(param_names), n=len(R))
stats, theta_map, AIC, BIC


## Plot Rotation Curve vs UFF (MAP)

In [ ]:

Rgrid = np.linspace(max(1e-3, np.min(R)), np.max(R), 200)
Vmap = v_circ_uff(Rgrid, theta_map)

plt.figure()
plt.errorbar(R, V, yerr=eV, fmt='o', label='Observed')
plt.plot(Rgrid, Vmap, label='UFF (MAP)')
plt.xlabel("R [kpc]")
plt.ylabel("V_circ [km/s]")
plt.title("UFF fit")
plt.legend()
plt.show()


## Posterior Predictive (quick-and-dirty bands)

In [ ]:

thetas = chain[:, :len(param_names)]
Rgrid = np.linspace(max(1e-3, np.min(R)), np.max(R), 200)
idxs = np.random.default_rng(0).integers(0, len(thetas), size=400)
Vpred = np.array([v_circ_uff(Rgrid, thetas[i]) for i in idxs])
lo, hi = np.percentile(Vpred, [16,84], axis=0)

plt.figure()
plt.errorbar(R, V, yerr=eV, fmt='o', label='Observed')
plt.plot(Rgrid, Vmap, label='UFF (MAP)')
plt.fill_between(Rgrid, lo, hi, alpha=0.3, label='Posterior band')
plt.xlabel("R [kpc]")
plt.ylabel("V_circ [km/s]")
plt.title("Posterior predictive")
plt.legend()
plt.show()
